In [3]:
from IPython.display import Image
import json
import os

![title](\imgs\cv_tsksk.png)

# Object detection

Задача:
найти границы объектов на изображении

![title](\imgs\label_test.jpg)

In [17]:
json_path = 'imgs\\test.json'
with open(json_path, "r") as read_file:
    data = json.load(read_file)
data['shapes'][0] # top left bottom right

{'label': 'dog',
 'points': [[22.762376237623762, 57.68316831683168],
  [164.84158415841583, 295.8019801980198]],
 'group_id': None,
 'shape_type': 'rectangle',
 'flags': {}}

1. Как представить выход сети?
2. Какие лоссы использовать?
3. Как мерить качество?

## R-CNN

1. Определение набора гипотез.
2. Извлечение из предполагаемых регионов признаков с помощью сверточной нейронной сети и их кодирование в вектор.
3. Классификация объекта внутри гипотезы на основе вектора из шага 2.
4. Улучшение (корректировка) координат гипотезы.
5. Все повторяется, начиная с шага 2, пока не будут обработаны все гипотезы с шага 1.

![title](\imgs\rcnn.jpg)

1. Отбор гипотез: генерируем из исходного изображения набор его частей (как мозайку). Для более осознаного выделения (чтобы в нарезанных частях были объекты) можно использовать Selective Search – верхнеуровнево, он позволяет составить набор гипотез (класс объекта пока не имеет значения), на основе сегментации определить границы объектов по интенсивности пикселей, перепаду цветов, контраста и текстур. Для более точной последующей обработки каждая гипотеза дополнительно расширяется на 16 пикселей во всех 4 направлениях – как бы добавляя контекст.
    
    Итог:

    Вход: исходное изображение.
    
    Выход: набор гипотез разного размера и соотношения сторон.

2. Извлечение из предполагаемых регионов признаков с помощью сверточной нейронной сети и их кодирование в вектор.

    Итог:

    Вход: каждая из предложенных на предыдущем шаге гипотеза.
    
    Выход: векторное представление для каждой гипотезы.

3. Классификация объекта внутри гипотезы на основе вектора из шага 2.

    Итог:

    Вход: вектор каждой из предложенных гипотез из предпоследнего слоя сети (в случае AlexNet это FC7).
    
    Выход: после последовательного запуска каждой гипотезы, получаем матрицу размерности $2000×N_{c}$, отображающую класс объекта для каждой гипотезы.

4. Улучшение (корректировка) координат гипотезы.
    
    Итог:

    Вход: карта признаков из последнего MaxPooling слоя для каждой гипотезы, которая содержит любой объект, кроме фона.
    
    Выход: поправки к координатам ограничивающей рамки гипотезы. 


В заключение выделим основные недостатки такого подхода:

1. Гипотезы, предложенные на шаге 1, могут частично дублировать друг друга – разные гипотезы могут состоять из одинаковых частей, а каждая такая гипотеза отдельно обрабатывалась нейронной сетью. Так получается, что большая часть запусков сети более или менее дублирует друг друга без надобности.

2. Нельзя использовать для real-time работы, поскольку на проход 1 изображения (кадра) тратится ~53 секунды (NVIDIA Titan Black GPU).

3. Алгоритм выделения гипотез никак не обучается, а поэтому дальнейшее улучшение качества почти невозможно (никто не отменял плохие гипотезы).


## Fast R-CNN

Процесс обработки изображения изменился и выглядит следующим образом:

1. Извлечение карты признаков изображения (не для каждой гипотезы по отдельности, а для всего изображения целиком);
2. Поиск гипотез (аналогично R-CNN на основе Selective Search);
3. Сопоставление каждой гипотезы с местом на карте признаков – т.е. использование единого набора выделенных признаков для каждой гипотезы (координаты гипотез можно однозначно сопоставить с местоположением на карте признаков);
4. Классификация каждой гипотезы и исправление координат ограничивающей рамки (эту часть стало возможным запускать параллельно, поскольку больше нет зависимости от SVM-классификации).

### RoI layer

В изначальной концепции R-CNN каждая предложенная гипотеза по отдельности обрабатывается с помощью CNN – такой подход стал своеобразным бутылочным горлышком. Для решения этой проблемы был разработан Region of Interest (RoI) слой. Этот слой позволяет единожды обрабатывать изображение целиком с помощью нейронной сети, получая на выходе карту признаков, которая далее используется для обработки каждой гипотезы.

Как подать на вход полносвязному слою гипотезы разного размера и соотношения сторон? Для этого и необходим RoI слой, который преобразовывает изображение с размерами $I_{h}×I_{w}$ в размеры $O_{h}×O_{w}$. Для этого необходимо исходное изображение разделить на сетку размером $O_{h}×O_{w}$ (размер ячейки примерно $\frac{I_{h}}{O_{h}}×\frac{I_{w}}{O_{w}}$) и из каждой ячейки выбрать максимальное число.

![title](\imgs\roi.jpg)

    Итог:

    Вход: координаты гипотезы и карта признаков изначального изображения;

    Выход: векторное представление гипотезы.

### Multi-task loss

В одновременном обучении сети для задач регрессии ограничивающих рамок и классификации применяется специальная лосс-функция:

$L(P,u,t^{u},v)=L_{cls}(P,u)+\lambda[u≥1]L_{loc}(t^{u},v)$


Здесь:

$\lambda$ необходим для настройки баланса между двумя функции (авторы использовали $\lambda$=1);
$u$ — правильный класс;
$L_{cls}$ представляет собой функции ошибки для классификации $L_{cls}(P,u)=-logP_{u}$;
$L_{loc}$ является SmoothL1-функцией и измеряет разницу между $v=(v_{x},v_{y},v_{w},v_{h})$ и $t^{u}=(t^u_x,t^u_y,t^u_w,t^u_h)$ значениями:

$SmoothL1=\left \{ \begin{matrix} \frac{1}{2}x^{2}, & if\left | x \right | <1\\ \left | x \right |-\frac{1}{2}, & otherwise \end{matrix}\right.$


Здесь, $x$ обозначает разность целевого значения и предсказания $t^u_i-v_{i}$. Такая функция сочетает в себе преимущества L1 и L2 функции, поскольку является устойчивой при больших значениях $x$ и не сильно штрафует при малых значениях.

## Faster R-CNN
![title](\imgs\rpn.jpeg)

Главная задача заменить Selective Search на что то более быстрое. Для этого был предложен

### RPN

![title](\imgs\faster.jpg)

1. Получим карту признаков $c×\frac{H}{16}×\frac{W}{16}$ с предыдущего шага.
2. Применим сверточный слой 3×3 (отступ равен единице – итоговая матрица не меняется в размерах). По всей видимости, этот шаг применяется авторами для дополнительного наращивания рецептивного поля ($P_{0}=106$, $r_{0}=228$).

* Ячейке ($i,j$) карты признаков соответствует вектор размерности $c$ (в нашем случае 512).

1. К каждому такому вектору применимы два сверточных слоя с ядром 1×1 и количеством выходных каналов $\hat{c}$ (ядро такого размера просто отображает размерность $c$ в $\hat{c}$):
    * Первой слой (cls) имеет параметр $\hat{c}=2k$ – необходим для определения вероятности наличия или отсутствия какого-либо объекта внутри гипотезы (классификация с 2 классами).
    * Второй слой (reg) имеет параметр $\hat{c}=4k$ – необходим для определения координат гипотез.

Отметим, что полученные вектора можно переформировать в матрицы $k×2$ и $k×4$. Таким образом получаем матрицы, где $i$ строке соответствуют значения для конкретной гипотезы.

Возникает вполне логичный вопрос, как из вектора, который поступает в reg слой можно определить абсолютные координаты гипотез? Ответ прост – никак. Для правильного определения координат необходимо использовать так называемые якоря (anchors) и поправки к их координатам.

Якорем называют четырехугольник разного соотношения сторон (1:1, 2:1, 1:2) и размеров (128×128, 256×256, 512×512). Центром якоря считается центр ячейки ($i,j$) карты признаков. Так, например, возьмем ячейку (7,7), центром которой являются значения (7.5,7.5), что соответствует координатам (120,120) исходного изображения (16×7.5). Сопоставим с этими координатами прямоугольники трех соотношений сторон и трех размеров (всего получается 3×3=9). В будущем слой reg по отношению к этим координатам будет выдавать соответствующие правки, тем самым корректируя местоположение и форму ограничивающей рамки.

    Итог:

    Вход: карта признаков изначального изображения;
    
    Выход: гипотезы, содержащие какой-либо объект.


### Loss функция

Для обучения RPN используется следующее обозначение классов:

Позитивными являются все якоря, имеющие пересечение (IoU) более 0.7 или имеющие наибольшее пересечение среди всех якорей (применяется в случае, если нет пересечения более 0.7).
Негативными являются все якоря, имеющие пересечение менее 0.3.
Все остальные якоря не участвуют в обучении (по сути, они являются нейтральными).

Таким образом класс $p^*_i$ якоря присуждается по следующему правилу:

$p^*_i = \begin{cases} 1 & if IoU > 0.7 \\ 0 & if IoU < 0.3 \\ nothing & otherwise \end{cases} $


С такими обозначениями минимизируется следующая функция:

$L(\{p_i\}, \{t_i\}) = \frac{1}{N_{cls}} \sum_i L_{cls}(p_i, p^*_i) + \lambda \frac{1}{N_{loc}} \sum_i p^*_i L_{reg} (t_i, t^*_i)$


Здесь:

$i$ – номер якоря;
$p_{i}$ – вероятность нахождения объекта в $i$ якоре;
$p^*_i$ – правильный номер класса (обозначен выше);
$t_{i}$ – 4 предсказанные поправки к координатам;
$t^*_i$ – ожидаемая (ground truth) поправки к координатам;
$L_{cls}(p_{i},p^*_i)$ – бинарный log-loss;
$L_{reg}(t_{i},t^*_i)$ – SmoothL1 лосс. Активируется только в случае $p^*_i=1$, т.е. если гипотеза содержит хоть какой-нибудь объект;
$\begin{Bmatrix}p_{i}\end{Bmatrix}$ и $\begin{Bmatrix}t_{i}\end{Bmatrix}$ – выходы классификационной и регрессионной модели соответственно;
$\lambda$ – коэффициент для настройки баланса между классификацией и регрессией.

Обе части комбинированного лосса нормализуются на $N_{cls}$ и $N_{loc}$ соответственно. Авторы использовали $N_{cls}$ равный размеру мини-батча (256), а $N_{loc}$ равный количеству якорей.

Для регрессии поправок к ограничивающим рамкам значения инициализируются и подсчитываются следующим образом:

$t_x = \frac{(x - x_a)}{w_a}, \quad\quad t^*_x = \frac{(x^*-x_a)}{w*} \\ t_y = \frac{(y - y_a)}{h_a}, \quad\quad t^*_y = \frac{(y^* - y_a)}{h_a} \\ t_w = \log{\frac{w}{w_a}}, \quad\quad t^*_w = \log{\frac{w^*}{w_a}} \\ t_h = \log{\frac{h}{h_a}}, \quad\quad t^*_h = \log{\frac{h^*}{h_a}}$


Здесь $x$, $y$, $w$ и $h$ обозначают центр, ширину и высоту ограничивающей рамки. Переменные $x$, $x^{*}$ и $x_{a}$ обозначают предсказание, ground truth и значение якорей (для $y$, $w$ и $h$ аналогично).

Обучение на полном списке якорей будет иметь смещение в сторону отрицательного класса (гипотез с таким классом в разы больше). В связи с этим мини-батч формируется в соотношении 1:1 позитивных якорей к негативным. В случае, если не удается найти соответствующее количество позитивных якорей, мини-батч дозаполняется с помощью негативных классов.

### Общее обучение сети

Основной задачей является совместное использование весов между двумя модулями –это повысит скорость работы. Поскольку невозможно (или довольно-таки сложно) обучить сразу два независимых модуля, авторы статьи используют итеративный подход:

1. Тренировка RPN сети. Сверточные слои инициализируются весами, ранее полученными при тренировке на ImageNet. Дообучаем на задаче определения регионов с каким-либо классом (уточнение класса занимается часть Fast R-CNN).
2. Тренировка Fast R-CNN сети. Так же, как и в п.1 инициализируем Fast R-CNN весами, ранее полученными при обучении на ImageNet. Дообучаем, используя гипотезы об объектах с помощью RPN сети, обученной в п.1. В этот раз задачей обучения является уточнение координат и определение конкретного класса объекта.
3. Используя веса из п.2 обучаем только RPN часть (слои, идущие до RPN сети, принадлежащие feature extractor, замораживаются и никак не изменяются).
4. Используя веса из п.3 (то есть, уже более точно настроенный RPN), обучаем слои для Fast R-CNN (остальные веса – идущие ранее или относящиеся к RPN — заморожены).

С помощью такого итеративного обучения получается, что вся сеть построена на одних и тех же весах. Можно и дальше обучать сеть по такому принципу, но авторы отмечают, что сильных изменений в метриках нет.

### Процесс предсказания

Во время использования нейронных сетей для предсказаний прохождение изображения выглядит так:

1. Изображение поступает на вход нейронной сети, генерируя карту признаков.
2. Каждая ячейка карты признаков обрабатывается с помощью RPN, выдавая в результате поправки к положению якорей и вероятность наличия объекта любого класса.
3. Соответствующие предсказанные рамки далее на основе карты признаков и RoI слоя поступают в дальнейшую обработку Fast R-CNN части.
4. На выходе получаем уже конкретный класс объектов и их точное положение на изображении.

In [ ]:
1. 